In [1]:
import pandas as pd 
import numpy as np 

In [12]:
df = pd.read_csv("../data/processed/features.csv")
df = df.drop(columns=['Unnamed: 0'])
df

,amount,hour,day_of_week,time_gap_minutes
0,2258,6,2,0.0
1,4884,12,2,6.0
2,5761,17,2,5.0
3,2080,21,2,4.0
4,499,10,3,13.0
...,...,...,...,...
495,589,8,6,2.0
496,2645,14,6,6.0
497,2048,7,0,17.0
498,2732,11,0,4.0


In [13]:
X = df
from sklearn.model_selection import train_test_split

X_train, X_test= train_test_split(X, test_size=0.2,random_state=42 )

In [14]:
# Fitting the scaler now 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train) # Computes mean and STD of each column

,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True


In [15]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Training the model
from sklearn.ensemble import IsolationForest
model = IsolationForest(random_state=42, contamination=0.1) # i believe 10% are abnormal  
model.fit(X_train_scaled)

,"n_estimators n_estimators: int, default=100The number of base estimators in the ensemble.",100
,"max_samples max_samples: ""auto"", int or float, default=""auto""The number of samples to draw from X to train each base estimator.- If int, then draw `max_samples` samples.- If float, then draw `max_samples * X.shape[0]` samples.- If ""auto"", then `max_samples=min(256, n_samples)`.If max_samples is larger than the number of samples provided,all samples will be used for all trees (no sampling).",'auto'
,"contamination contamination: 'auto' or float, default='auto'The amount of contamination of the data set, i.e. the proportionof outliers in the data set. Used when fitting to define the thresholdon the scores of the samples.- If 'auto', the threshold is determined as in the original paper.- If float, the contamination should be in the range (0, 0.5]... versionchanged:: 0.22 The default value of ``contamination`` changed from 0.1 to ``'auto'``.",0.1
,"max_features max_features: int or float, default=1.0The number of features to draw from X to train each base estimator.- If int, then draw `max_features` features.- If float, then draw `max(1, int(max_features * n_features_in_))` features.Note: using a float number less than 1.0 or integer less than number offeatures will enable feature subsampling and leads to a longer runtime.",1.0
,"bootstrap bootstrap: bool, default=FalseIf True, individual trees are fit on random subsets of the trainingdata sampled with replacement. If False, sampling without replacementis performed.",False
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for :meth:`fit`. ``None`` means 1unless in a :obj:`joblib.parallel_backend` context. ``-1`` means usingall processors. See :term:`Glossary ` for more details.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseudo-randomness of the selection of the featureand split values for each branching step and each tree in the forest.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",42
,"verbose verbose: int, default=0Controls the verbosity of the tree building process.",0
,"warm_start warm_start: bool, default=FalseWhen set to ``True``, reuse the solution of the previous call to fitand add more estimators to the ensemble, otherwise, just fit a wholenew forest. See :term:`the Glossary `... versionadded:: 0.21",False


In [25]:
import joblib

joblib.dump(model, "../models/anomaly_model.pk")
joblib.dump(scaler, "../models/scaler.pk")

['../models/scaler.pk']

## Model Validation 

In [28]:
model.predict(X_test_scaled)


array([ 1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1, -1,
        1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1])

In [30]:
model.decision_function(X_train_scaled) 
# Higher score → more normal
# Lower score → more abnormal

array([ 4.63148488e-02, -8.26079705e-04,  9.89289762e-02,  8.50944640e-02,
        7.63440287e-02,  9.97507474e-02,  7.79617351e-02,  1.26394471e-01,
        1.19776600e-01,  4.15758859e-02,  1.11846132e-01, -5.07858442e-02,
        5.46446408e-02,  7.21226596e-02,  3.03861194e-02,  1.35231293e-01,
        5.71050876e-02,  4.03942476e-02,  1.31380742e-01,  1.32243064e-01,
        6.32425952e-02,  1.20419768e-01, -2.18415870e-02,  4.94087087e-02,
        3.61292868e-02,  1.24931347e-01,  7.60788430e-02,  1.24312279e-01,
        9.61880121e-02,  9.32391353e-02,  3.41776071e-03,  2.02266041e-02,
       -1.02088434e-02,  5.74215039e-02,  5.86628773e-02,  7.50496198e-02,
        8.77736681e-02,  8.47770653e-02,  4.99706051e-02,  5.09378803e-02,
        3.12473386e-02,  1.15709541e-02,  7.60557504e-02,  1.25261581e-01,
       -3.47571869e-02,  7.79922707e-02,  4.74426431e-02,  7.53081488e-03,
        7.39229789e-02,  1.31380682e-01,  1.12844463e-01,  8.24227678e-02,
        6.41563931e-02,  

In [34]:
# Sense of model 
df_train = X_train.copy()
df_train['prediction'] = model.predict(X_train_scaled)
df_train.head(10)

,amount,hour,day_of_week,time_gap_minutes,prediction
249,16504,9,0,3.0,1
433,18236,2,6,1.0,-1
19,690,11,0,7.0,1
322,1087,11,0,10.0,1
332,482,10,4,23.0,1
56,701,13,0,2.0,1
301,2027,19,3,1.0,1
229,3845,15,3,5.0,1
331,4321,11,3,17.0,1
132,24153,1,3,2.0,1


In [ ]:
df_train['prediction'].value_counts() # Contamination

prediction
 1    360
-1     40
Name: count, dtype: int64

In [37]:
# Get top anomalies 
anomalies = df_train[df_train['prediction'] == -1]
anomalies

,amount,hour,day_of_week,time_gap_minutes,prediction
433,18236,2,6,1.0,-1
423,44887,3,4,1.0,-1
320,1916,20,6,25.0,-1
470,16675,12,6,17.0,-1
253,23758,4,2,27.0,-1
462,46749,4,3,8.0,-1
453,38442,4,0,1.0,-1
390,21566,2,0,0.0,-1
275,30718,0,3,0.0,-1
210,2656,6,3,57.0,-1
